<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Ответьте-на-вопрос-и-обоснуйте-решение" data-toc-modified-id="Ответьте-на-вопрос-и-обоснуйте-решение-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Ответьте на вопрос и обоснуйте решение</a></span></li><li><span><a href="#Математическое-доказательство" data-toc-modified-id="Математическое-доказательство-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Математическое доказательство</a></span></li><li><span><a href="#Экспериментальное-доказательство" data-toc-modified-id="Экспериментальное-доказательство-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Экспериментальное доказательство</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Описание данных

Набор данных находится в файле `/datasets/insurance.csv`. 

*	Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
*	Целевой признак: количество страховых выплат клиенту за последние 5 лет.


## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
#
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [2]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except FileNotFoundError:
    df = pd.read_csv('insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
df.isnull().sum().sort_values(ascending=False) / df.shape[0] * 100  # процент пропусков в наборе данных по столбцам

Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64

Пропусков в данных нет, и это хорошо. Тип данных у возраста и зар. платы менять не будем, и то и другое может содержать дробные части. Названия столбцов приведем в соответствие к змеиному стилю

### Предобработка данных

In [7]:
cols_name = ('gender', 'age', 'salary', 'members_family', 'insurance')

In [8]:
df = df.set_axis(cols_name, axis='columns')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gender          5000 non-null   int64  
 1   age             5000 non-null   float64
 2   salary          5000 non-null   float64
 3   members_family  5000 non-null   int64  
 4   insurance       5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Отлично, проверим на наличие дубликатов

In [10]:
df.duplicated().sum()

153

Удалим дубликаты

In [11]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gender          4847 non-null   int64  
 1   age             4847 non-null   float64
 2   salary          4847 non-null   float64
 3   members_family  4847 non-null   int64  
 4   insurance       4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 227.2 KB


Подготовку данных завершили, можно двигаться дальше

## Умножение матриц

### Ответьте на вопрос и обоснуйте решение



В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 

* a. Изменится. Приведите примеры матриц.
* b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.


Вспомнив первый курс Мат-Меха можно отметить следующее - Матрица имеет обратную матрицу тогда и только тогда, когда она квадратная и невырожденная, то есть ее определитель отличен от нуля. Такая матрица называется обратимой. Если обратная матрица существует, то она одна

При умножении исходной матрицы признаков $X$ на обратимую матрицу соответствующего размера качество линейно регрессии не изменится. Проверим, взяв случайную обратимую матрицу и обучив модель линейной регрессии на исходных данных $X$ и получившися после умножения $X$ на обратимую матрицу.


Для начала преобразуем наши данные, отделим признаки и целевой признак, и добавим к матрице признаков единичный столбец вначале

In [12]:
X = df.drop(['insurance'], axis=1)
y = df['insurance']

In [13]:
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

Проверим размерности

In [14]:
print(X.shape,y.shape)

(4847, 5) (4847,)


Сформируем случайную матрицу $P$, размерности $5x5$

In [15]:
def rnd_matrix(n):
    random_matrix = np.random.normal(size=(n, n))
    try:
        np.linalg.inv(random_matrix) # выполняется операция создания обратной матрицы
    except LinAlgError: # если возникает ошибка, т.е. обратной матрицы не существует, выполняется еще одна попытка
        rnd_matrix(n)
    return random_matrix

In [25]:
P = rnd_matrix(5)
P

array([[ 0.26458224, -0.89033984,  1.76017909,  2.22048163, -0.36590787],
       [ 1.05057794, -0.47455868,  0.51970463, -0.10000394, -0.63937907],
       [ 0.76593628,  1.41298745, -0.93269494, -0.37533063, -0.59664865],
       [ 0.49024573, -1.04250473, -0.94017995,  1.3060236 ,  1.18680771],
       [ 0.39372819,  0.57799912, -0.44735129, -0.35779747, -1.03674047]])

Посчитаем детерминант матрицы  $P$

In [26]:
np.linalg.det(P)

-7.063394837640996

Определитель отличен от нуля, значит матрица имеет обратную

In [27]:
P_inv = np.linalg.inv(P)
P_inv

array([[-0.03405406,  0.74674684,  0.57416375,  0.0971508 , -0.66773545],
       [ 0.1786302 , -0.24763629,  0.81744741, -0.27981325, -0.701084  ],
       [ 0.16394804,  0.30097634,  0.24856833, -0.42273046, -0.87045466],
       [ 0.37724155, -0.40144535,  0.14729555,  0.20982559,  0.26986411],
       [-0.11427969,  0.15420996,  0.51570188, -0.00911208, -1.32655175]])

In [28]:
np.linalg.det(P_inv)

-0.14157498242501987

Произведение матрицы на обратную к этой матрице равно единичной матрице
$A * A^{-1} = E$

Проверим

In [29]:
P.dot(P_inv)

array([[ 1.00000000e+00,  9.18830160e-18,  2.56539987e-17,
         6.11423874e-17,  1.29392040e-16],
       [ 3.26287103e-18,  1.00000000e+00, -7.81133477e-18,
         7.85919554e-18,  2.77445610e-16],
       [-3.22461178e-17,  3.44312851e-17,  1.00000000e+00,
        -5.28213718e-17,  4.16985417e-17],
       [ 1.53056856e-17,  5.65933350e-17, -9.64115247e-17,
         1.00000000e+00,  1.01139281e-16],
       [ 3.96110060e-18,  1.27190184e-18,  1.02203555e-16,
        -2.11785998e-17,  1.00000000e+00]])

Можно заметить что на главной диагонали единицы, в остальных местах бесконечно малые числа, если округлить полуим нули

Единичная матрица так же обратимая: поскольку является квадратной и ее определитель равен 1. Обозначается $E$. Матрица обратная к единичной так же единичная. Так же произведение любой матрица на единичную соответствующего размера равно исходной матрице:
$AE=EA=A$

In [30]:
E = np.eye(5)
E

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [31]:
E_inv = np.linalg.inv(E)
E_inv

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

###  Математическое доказательство
Для доказательства того, что при умножении матрицы $X$ на обратимую матрицу $P$ метрики линейной регрессии не изменяться, подставим в формулу 
$$
w = (X^T X)^{-1} X^T y
$$ вместо $X$ матрицу $XP$ и посмотрим какая в итоге формула получится

$$
w = ((XP)^T XP)^{-1} (XP)^T y
$$
Раскроем скобки у транспонирования
$$
w = (P^T X^T XP)^{-1} P^TX^T y
$$
Раскроем скобки для операции произведения обратных матриц
$$
w = (XP)^{-1}(P^T X^T)^{-1} P^TX^T y
$$
Еще раз
$$
w = P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^T y
$$
Учитывая что для обратимой матрицы $PP^{-1} = E$ и так же $(P^T)^{-1}(P^T) = E$
$$
w = P^{-1}(X^TX)^{-1} E X^T y
$$
или
$$
w = P^{-1}(X^TX)^{-1} X^T y
$$

Подставим получившуюся формулу в формулу предсказания
$a=Xw$ в которой $X$ заменим на $XP$

$$
a = XP P^{-1}(X^TX)^{-1} X^T y
$$
Учитывая что для обратимой матрицы $PP^{-1} = E$ и так же $(P^T)^{-1}(P^T) = E$
$$
a = X(X^TX)^{-1} X^T y
$$

Не трудно заметить, что если в исходную формулу $a=Xw$ подставить исходную формулу $w = (X^T X)^{-1} X^T y$ то получим тоже выражение
$$
a = X(X^TX)^{-1} X^T y
$$

Следовательно умножение матриы признаков $X$ на обратимую матрицу $P$ не меняет формулы для предсказаний, и не поменяет метрик модели линейной регрессии

Доказали, теперь проверим!
Обучим две моедли линейной регрессии на исходной матрице признаков $X$ на измененной матрице $XP$

### Экспериментальное доказательство

In [32]:
X_P = X.dot(P)
X_P_inv = X.dot(P_inv) 

In [33]:
def linear_model(X,y,matrix_name):
    model = LinearRegression()
    model.fit(X,y)
    predict = model.predict(X)
    return print("Метрика R2 для матрицы",matrix_name,"равна",round(r2_score(y,predict),6))

In [34]:
linear_model(X,y,'X')

Метрика R2 для матрицы X равна 0.430201


In [35]:
linear_model(X_P,y,'XP')

Метрика R2 для матрицы XP равна 0.430202


Эксперимент подтверждил выкладки приведенные выше

## Алгоритм преобразования

Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

**Алгоритм и обоснование**

1. Для преобразования данных клиентов перед машинным обучением, можно умножать матрицу признаков на обратимую матрицу, выше уже было доказано, что при таком преобразовании предсказания модели и метрики не поменяются.


2. Можно так же все признаки обработать алгоритмом StandartScaler, после него так же не получится восстановить перс данные 


## Проверка алгоритма

Домножение на обратимую матрицу мы уже проверяли, посмотрим теперь на вариант с методом StandartScaler

In [36]:
scaler = StandardScaler()
X_S = scaler.fit_transform(X)

In [37]:
linear_model(X_S,y,'X scaler')

Метрика R2 для матрицы X scaler равна 0.430201


In [38]:
linear_model(X,y,'X')

Метрика R2 для матрицы X равна 0.430201


In [39]:
linear_model(X_P,y,'XP')

Метрика R2 для матрицы XP равна 0.430202


Все три метрики равны

Посмотрим как меняется матрица признаков после преобразований

In [40]:
X

array([[1.00e+00, 1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [1.00e+00, 0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [1.00e+00, 0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [1.00e+00, 0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

In [41]:
X_P

array([[ 24349.30029362, -51651.08925195, -46669.33356056,
         64765.14482008,  58839.15780686],
       [ 18665.22897362, -39550.49483429, -35768.42930106,
         49613.4943791 ,  45069.84450363],
       [ 10317.63698386, -21852.51313122, -19769.06695822,
         27417.83155078,  24905.29319673],
       ...,
       [ 16635.70088119, -35312.38509676, -31889.88878227,
         44268.19840685,  40218.40901603],
       [ 16050.38218849, -34058.44999987, -30763.46587686,
         42699.76162089,  38791.37034687],
       [ 19927.13158648, -42286.91547704, -38195.58896174,
         53015.81169367,  48165.64484698]])

In [42]:
X_S

array([[ 0.        ,  1.0030995 ,  1.1754362 ,  0.97315092, -0.18517565],
       [ 0.        , -0.99691008,  1.76456423, -0.19011493, -0.18517565],
       [ 0.        , -0.99691008, -0.23847105, -1.89490109, -1.09546611],
       ...,
       [ 0.        , -0.99691008, -1.29890149, -0.60126924,  0.7251148 ],
       [ 0.        ,  1.0030995 , -1.06325028, -0.72160708,  1.63540526],
       [ 0.        ,  1.0030995 , -0.35629665,  0.07061707, -0.18517565]])

По преобразованным матрицам получить исходные персональные данные не получится, если конечно не знать матрицу преобразований $P$

In [43]:
X_P.dot(P_inv)

array([[ 1.00000000e+00,  1.00000000e+00,  4.10000000e+01,
         4.96000000e+04,  1.00000000e+00],
       [ 1.00000000e+00,  4.69533575e-12,  4.60000000e+01,
         3.80000000e+04,  1.00000000e+00],
       [ 1.00000000e+00, -3.20866837e-13,  2.90000000e+01,
         2.10000000e+04,  1.26302805e-12],
       ...,
       [ 1.00000000e+00, -1.08381723e-12,  2.00000000e+01,
         3.39000000e+04,  2.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00,  2.20000000e+01,
         3.27000000e+04,  3.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00,  2.80000000e+01,
         4.06000000e+04,  1.00000000e+00]])

Так можно получить обратнно, исходные данные

## Вывод

В этой работе, мы предложили два варианта обработки исходных данных клиентов страховой компании для защиты их персональных данных.

Обосновали сначала теоретически возможность применения, а после показали на практике что метрика модели не изменяется после преобразования исходных данных.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования